# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [2]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [3]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [14]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [17]:
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [8]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [33]:
tmp = sales.groupby('ProductID')['Qty'].sum() + sales.groupby('ProductID')['Amt'].sum()
tmp

ProductID
p1001771     3355882
p1002841    11012444
p1005621     2602609
p1005771     7323926
p1005891    10124231
              ...   
p1246581     2458526
p1255281      956442
p1256521     1242246
p1284851     6439673
p1299491     1094849
Length: 62, dtype: int64

### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [36]:
tmp.sort_values(ascending = False).head()

ProductID
p1072601    18133125
p1178011    14080471
p1002841    11012444
p1005891    10124231
p1194801     7518654
dtype: int64

### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [38]:
customers.groupby('Gender').sum()

,BirthYear
Gender,
F,4060490
M,360704


### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [39]:
customers['Age'] = 2022-customers['BirthYear']

### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [43]:
bin = [-np.inf,30,40,50,60,70,np.inf]
customers['AgeGroup'] = pd.cut(customers['Age'],bins=bin,labels = ['30미만','30대','40대','50대','60대','70이상'])

### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [46]:
customers.groupby('AgeGroup')['CustomerID'].count()

AgeGroup
30미만      9
30대     236
40대     966
50대     696
60대     242
70이상     94
Name: CustomerID, dtype: int64

### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [47]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
temp = pd.merge(sales,products,on='ProductID',how='inner')

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1
1,137,4,2016-01-02,p1036481,2,2100,c280590,순두부,반찬류,두부,1
2,63,16,2016-01-03,p1036481,1,1050,c037915,순두부,반찬류,두부,1
3,135,3,2016-01-04,p1036481,3,3150,c100815,순두부,반찬류,두부,1
4,63,13,2016-01-06,p1036481,10,10500,c048405,순두부,반찬류,두부,1


In [51]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.
temp2 = temp.groupby('ProductName')['Amt'].sum()


In [52]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.
temp2.sort_values(ascending = False).head()


ProductName
우유1000    18129067
사과_소      14078818
딸기_대      11011541
두부_대      10119037
딸기_소       7517664
Name: Amt, dtype: int64

### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [53]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.
temp3 = pd.merge(sales,customers,on='CustomerID',how = 'inner')


In [54]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.
temp3.groupby('AgeGroup')['Amt'].sum()


AgeGroup
30미만      283767
30대     15894418
40대     90520765
50대     63994308
60대     18295892
70이상     6010318
Name: Amt, dtype: int64

### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [55]:
# 1) 세 데이터프레임을 merge 
a = pd.merge(sales,products,on='ProductID',how='inner')
total = pd.merge(a,customers,on='CustomerID',how='inner')


In [81]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다
for i in age:
    print(i)
    print((total.loc[total['AgeGroup']==i].groupby('Category')['Amt'].sum()/ #그 연령대의 카테고리별 매출액 합
           (total.loc[total['AgeGroup']==i]['Amt'].sum())))# 그 연령대의 전체 매출액 합


30미만
Category
간식     0.128979
과일     0.218137
반찬류    0.239348
유제품    0.140658
채소     0.272879
Name: Amt, dtype: float64
30대
Category
간식     0.065935
과일     0.269342
반찬류    0.118283
유제품    0.274941
채소     0.271499
Name: Amt, dtype: float64
40대
Category
간식     0.082607
과일     0.260447
반찬류    0.141450
유제품    0.252397
채소     0.263098
Name: Amt, dtype: float64
50대
Category
간식     0.056672
과일     0.250408
반찬류    0.175663
유제품    0.225107
채소     0.292151
Name: Amt, dtype: float64
60대
Category
간식     0.032321
과일     0.229957
반찬류    0.249359
유제품    0.160552
채소     0.327811
Name: Amt, dtype: float64
70이상
Category
간식     0.023342
과일     0.272638
반찬류    0.274355
유제품    0.110113
채소     0.319553
Name: Amt, dtype: float64
